In [1]:
import numpy  as np
from pymks.stats import correlate 

In [2]:
    >>> from pymks import PrimitiveBasis
    >>> prim_basis = PrimitiveBasis(2, [0, 1])
    >>>
    >>> np.random.seed(0)
    >>> X = np.random.randint(2, size=(1, 3))
    >>> X_corr = correlate(X, prim_basis)
    >>> X_result = np.array([[0, 0.5, 0],
    ...                      [1 / 3., 2 / 3., 0],
    ...                      [0, 0.5, 0.5]])
    >>> assert np.allclose(X_corr, X_result)

[0 1]


In [3]:
def _auto_correlations(n_states):
    """Returns list of autocorrelations

    Args:
        n_states: number of local states

    Returns:
        list of tuples for autocorrelations

    >>> l = _auto_correlations(np.arange(3))
    >>> assert l == ((0, 1, 2), (0, 1, 2))
    """
    n_states=np.arange(n_states)
    return tuple(n_states), tuple(n_states)


def _cross_correlations(n_states):
    """Returns list of crosscorrelations

    Args:
        n_states: number of local states

    Returns:
        list of tuples for crosscorrelations

    >>> l = _cross_correlations(np.arange(3))
    >>> assert l == ((0, 0, 1), (1, 2, 2))
    """
    n_states=np.arange(n_states)
    l = range(len(n_states))
    cross_corr = [[(l[i], l[j]) for j in l[1:][i:]] for i in l[:-1]]
    flat_corr = [item for sublist in cross_corr for item in sublist]
    l_0 = tuple([_l[0] for _l in flat_corr])
    l_1 = tuple([_l[1] for _l in flat_corr])
    return l_0, l_1


In [4]:
# First Get the number of states
L=3
_auto, _cross = _auto_correlations(L), _cross_correlations(L)
correlations = (_auto[0] + _cross[0], _auto[1] + _cross[1])


In [5]:
correlations


((0, 1, 2, 0, 0, 1), (0, 1, 2, 1, 2, 2))

In [6]:
l2=_cross_correlations(3)

## 

In [7]:
l2

((0, 0, 1), (1, 2, 2))

In [8]:
def _correlations_to_indices(correlations, n_states):
    """
    Helper function to select correct indices given the local state values in
    basis.n_states.

    Args:
        correlations: list of correlations to be computed
        basis: an instance of a basis class.

    Returns:
        list of correlations in terms of indices
    """   
    n_states=np.arange(n_states)
    try:
        l_0 = tuple([list(n_states).index(_l[0]) for _l in correlations])
        l_1 = tuple([list(n_states).index(_l[1]) for _l in correlations])
    except ValueError as ve:
        raise ValueError('correlations value ' + str(ve) +
                         ' is not in basis.n_states')
    return (l_0, l_1)

In [9]:

correlations=[(0,0),(0,1),(1,1)]


a=_correlations_to_indices(correlations,3)

In [10]:
a

((0, 0, 1), (0, 1, 1))

In [11]:
correlations2=[(0, 0), (1, 1)]

In [12]:
type(correlations2)

list

In [13]:
type(correlations2[0])

tuple

In [14]:
import numpy as np
import dask.array as da
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from pymks.fmks.correlations import FlattenTransformer, TwoPointcorrelation
from pymks.fmks.data.cahn_hilliard import generate
from pymks.fmks.bases.legendre import LegendreTransformer


def test_classification():
    """This test basically creates Legendre microstructures in both times:
    0 and t.  Then builds homogenization classification linkages to
    classify if newly generated microstructures are at time 0 or t
    """
    reducer = PCA(n_components=3)
    linker = LogisticRegression()
    homogenization_pipeline = Pipeline(
        steps=[
            ("discretize", LegendreTransformer(n_state=3, min_=-1.0, max_=1.0)),
            (
                "Correlations",
                TwoPointcorrelation(
                    periodic_boundary=True, cutoff=10, correlations1=1, correlations2=1
                ),
            ),
            ("flatten", FlattenTransformer()),
            ("reducer", reducer),
            ("connector", linker),
        ]
    )
    da.random.seed(3)
    x0_phase, x1_phase = generate(shape=(50, 21, 21))
    y0_class = np.zeros(x0_phase.shape[0])
    y1_class = np.ones(x1_phase.shape[0])
    x_combined = np.concatenate((x0_phase, x1_phase))
    y_combined = np.concatenate((y0_class, y1_class))
    homogenization_pipeline.fit(x_combined, y_combined)
    x0_test, x1_test = generate(shape=(3, 21, 21))
    y1_test = homogenization_pipeline.predict(x1_test)
    y0_test = homogenization_pipeline.predict(x0_test)
    assert np.allclose(y0_test, [0, 0, 0])
    assert np.allclose(y1_test, [1, 1, 1])


test_classification()

TypeError: __init__() got an unexpected keyword argument 'correlations1'

In [15]:
## Berkay Use correlations to indices
import numpy as np
import dask.array as da
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from pymks.fmks.correlations import FlattenTransformer, TwoPointcorrelation
from pymks.fmks.data.cahn_hilliard import generate
from pymks.fmks.bases.legendre import LegendreTransformer
x0_phase, x1_phase = generate(shape=(50, 21, 21))



In [16]:
a=LegendreTransformer(n_state=3, min_=-1.0, max_=1.0).transform(x0_phase).compute()

In [17]:
a.shape

(50, 21, 21, 3)

In [18]:
from pymks.fmks.correlations import two_point_stats


In [19]:
c=TwoPointcorrelation(periodic_boundary=True, cutoff=None, correlations=[(0,0),(1,1)]).transform(a)

In [21]:
c.shape

(50, 15, 15, 2)

In [ ]:
from toolz import pipe

In [ ]:
np.zeros((2,2,2)).shape

In [ ]:
correl=np.ndarray([])

In [ ]:
correl[0,:,:,:]=np.zeros((2,2,2))

In [ ]:
correlations=[(0,0),(0,1)]
len(correlations)

In [ ]:
for Cor in correlations:
    print(Cor[0],Cor[1])

In [ ]:
for Cor in correlations:
    correl=[]
    br=pipe(
        a,
        lambda x: da.from_array(x, chunks=x.shape),
        lambda x: (x[..., Cor[0]], x[..., Cor[1]]),
        lambda x: two_point_stats(
            *x, periodic_boundary=True, cutoff=8
        ),
    )

In [ ]:
br.compute().shape[2]

In [ ]:
a.shape[2]

In [ ]:
np.empty((10,10))

In [ ]:
b=TwoPointcorrelation(periodic_boundary=True, cutoff=None, correlations=[(1,1)]).transform(a)

In [ ]:
print(b)

In [ ]:
a.shape()

In [ ]:
correlations1=0
correlations2=0
correlations=[(0,0),(1,1)]


In [ ]:
from pymks.fmks.func import rcompose

In [ ]:
def cor_assign(cor):
    corr1=cor[0]
    corr2=cor[1]
    return corr1,corr2
    
cor_assign(correlations)

In [ ]:
from toolz.curried import curry
from toolz.curried import map as map_

In [ ]:
yy=[(1,1),(2,1)]

@curry
def add(i, x):
    return i + x[0] + x[1]

def calc(i, values):
    return map(add(i), values)
#rcompose(lambda x: 2*x,map(lambda x:x+yy[0]+yy[1])
 
print(list(calc(3, yy)))
         
        

In [ ]:
yy=[(1,1),(2,1)]

@curry
def add(i, x):
    return i + x[0] + x[1]

def calc(i, values):
    return pipe(
        values,
        map_(add(i))
    )
#rcompose(lambda x: 2*x,map(lambda x:x+yy[0]+yy[1])
 
print(list(calc(3, yy)))
         
        

In [ ]:
import numpy as np

aa = list(map_(lambda _: np.random.random(150).reshape((5, 6, -1)), range(4)))
np.stack(aa, axis=-1).shape

In [ ]:


rcompose(lambda x: da.from_array(x, chunks=x.shape),
            map(lambda x,correlations: (x[..., correlations[0]], x[..., correlations[1]]),correlations),
            lambda x: two_point_stats(
                *x, periodic_boundary=True, cutoff=None
            ),)(a)

In [ ]:
b=pipe(
            a,
            lambda x: da.from_array(x, chunks=x.shape),
            lambda x: (x[..., correlations[0]], x[..., correlations[1]]),
            lambda x: two_point_stats(
                *x, periodic_boundary=True, cutoff=None
            ),
        )




In [ ]:
b.compute()